In [2]:
import pandas as pd
import requests
import json
import time
from pandas.io.json import json_normalize

In [12]:
# df = pd.read_csv('./birthdays.csv', parse_dates=['Birth Date'])
df = pd.read_csv('./birthdays4.csv')
df.rename(columns={'Birth Date':"birth_date"}, inplace=True)
df.head()

,Latitude,Longitude,birth_date
0,49.535120,-111.631584,1534735374
1,31.619696,-109.441666,1553185679
2,32.371845,-98.762950,1529876532
3,30.327080,-92.548600,1521882642
4,44.542300,-82.181320,1541924312


In [13]:
df["birth_date"] = pd.to_datetime(df['birth_date'], unit='s')
df

,Latitude,Longitude,birth_date
0,49.535120,-111.631584,2018-08-20 03:22:54
1,31.619696,-109.441666,2019-03-21 16:27:59
2,32.371845,-98.762950,2018-06-24 21:42:12
3,30.327080,-92.548600,2018-03-24 09:10:42
4,44.542300,-82.181320,2018-11-11 08:18:32
...,...,...,...
995,33.577890,-119.107864,2019-02-08 10:38:52
996,48.951250,-102.614150,2018-05-27 05:24:01
997,42.429970,-88.156440,2018-01-10 04:27:05
998,43.224995,-92.340090,2018-11-21 19:32:35


In [14]:
df['maxtempC'] = 0
df['mintempC'] = 0
df['totalSnow_cm'] = 0.0
df['sunHour'] = 0.0
df['tempC'] = 0.0

df['windspeedKmph'] = 0.0
df['winddirDegree'] = 0.0
df['precipMM'] = 0.0
df['humidity'] = 0.0
df['visibility'] = 0.0

df['pressure'] = 0
df['cloudcover'] = 0.0
df['HeatIndexC'] = 0.0
df['DewPointC'] = 0.0

df['WindChillC'] = 0.0
df['WindGustKmph'] = 0.0
df['FeelsLikeC'] = 0.0
df['uvIndex'] = 0

df['sunrise'] = ""
df['sunset'] = ""

df.head()

,Latitude,Longitude,birth_date,maxtempC,mintempC,totalSnow_cm,sunHour,tempC,windspeedKmph,winddirDegree,...,pressure,cloudcover,HeatIndexC,DewPointC,WindChillC,WindGustKmph,FeelsLikeC,uvIndex,sunrise,sunset
0,49.535120,-111.631584,2018-08-20 03:22:54,0,0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0,,
1,31.619696,-109.441666,2019-03-21 16:27:59,0,0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0,,
2,32.371845,-98.762950,2018-06-24 21:42:12,0,0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0,,
3,30.327080,-92.548600,2018-03-24 09:10:42,0,0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0,,
4,44.542300,-82.181320,2018-11-11 08:18:32,0,0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0,,


In [15]:
def hourlyIndex(hour):
    if hour >= 0 and hour < 3:
        return 0
    elif hour >= 3 and hour < 6:
        return 1
    elif hour >= 6 and hour < 9:
        return 2
    elif hour >= 9 and hour < 12:
        return 3
    elif hour >= 12 and hour < 15:
        return 4
    elif hour >= 15 and hour < 18:
        return 5
    elif hour >= 18 and hour < 21:
        return 6
    else:
        return 7

In [ ]:
for row in df.itertuples():
    print("fetching for row " + str(row.Index))
    date = row.birth_date.strftime('%Y-%m-%d')
    lat = row.Latitude
    lon = row.Longitude
    url = 'http://api.worldweatheronline.com/premium/v1/past-weather.ashx?q=%s,%s&date=%s&format=json&key=95ef0cde25e24de58f5142006201002' %(lat, lon, date)
    
    try:
        r = requests.get(url)
        weather = r.json()
        
        maxtempC = weather['data']['weather'][0]['maxtempC']
        mintempC = weather['data']['weather'][0]['mintempC']
        totalSnow = weather['data']['weather'][0]['totalSnow_cm']
        sunhour = weather['data']['weather'][0]['sunHour']
        sunrise = weather['data']['weather'][0]['astronomy'][0]['sunrise']
        sunset = weather['data']['weather'][0]['astronomy'][0]['sunset']
        
        hourlyIdx = hourlyIndex(row.birth_date.hour)
        if hourlyIdx >= len(weather['data']['weather'][0]['hourly']):
            hourlyIdx = 0
        
        tempC = weather['data']['weather'][0]['hourly'][hourlyIdx]['tempC']
        windspeedKmph = weather['data']['weather'][0]['hourly'][hourlyIdx]['windspeedKmph']
        winddirDegree = weather['data']['weather'][0]['hourly'][hourlyIdx]['winddirDegree']
        precipMM = weather['data']['weather'][0]['hourly'][hourlyIdx]['precipMM']
        humidity = weather['data']['weather'][0]['hourly'][hourlyIdx]['humidity']
        visibility = weather['data']['weather'][0]['hourly'][hourlyIdx]['visibility']
        pressure = weather['data']['weather'][0]['hourly'][hourlyIdx]['pressure']
        cloudcover = weather['data']['weather'][0]['hourly'][hourlyIdx]['cloudcover']
        HeatIndexC = weather['data']['weather'][0]['hourly'][hourlyIdx]['HeatIndexC']
        DewPointC = weather['data']['weather'][0]['hourly'][hourlyIdx]['DewPointC']
        WindChillC = weather['data']['weather'][0]['hourly'][hourlyIdx]['WindChillC']
        WindGustKmph = weather['data']['weather'][0]['hourly'][hourlyIdx]['WindGustKmph']
        FeelsLikeC = weather['data']['weather'][0]['hourly'][hourlyIdx]['FeelsLikeC']
        uvIndex = weather['data']['weather'][0]['hourly'][hourlyIdx]['uvIndex']
   
        df.at[row.Index, "maxtempC"] = maxtempC
        df.at[row.Index, "mintempC"] = mintempC
        df.at[row.Index, "totalSnow_cm"] = totalSnow
        df.at[row.Index, "sunHour"] = sunhour
        df.at[row.Index, "tempC"] = tempC
        
        df.at[row.Index, "windspeedKmph"] = windspeedKmph
        df.at[row.Index, "winddirDegree"] = winddirDegree
        df.at[row.Index, "precipMM"] = precipMM
        df.at[row.Index, "humidity"] = humidity
        df.at[row.Index, "visibility"] = visibility
        
        df.at[row.Index, "pressure"] = pressure
        df.at[row.Index, "cloudcover"] = cloudcover
        df.at[row.Index, "HeatIndexC"] = HeatIndexC
        df.at[row.Index, "DewPointC"] = DewPointC
        
        df.at[row.Index, "WindChillC"] = WindChillC
        df.at[row.Index, "WindGustKmph"] = WindGustKmph
        df.at[row.Index, "FeelsLikeC"] = FeelsLikeC
        df.at[row.Index, "uvIndex"] = uvIndex
        
        df.at[row.Index, "sunrise"] = sunrise
        df.at[row.Index, "sunset"] = sunset
    except:
        print("json decode error on: " + str(row.Index))
        print(weather)

In [19]:
def timestampToSeason(ts):
    month = ts.month
    if month >= 3 and month < 6:
        return 'spring'
    elif month >= 6 and month < 9:
        return 'summer'
    elif month >= 9 and month < 12:
        return 'autumn'
    else:
        return 'winter'

df = df[df.sunHour != 0] # if sunHour is zero, data is invalid
df['season'] = df["birth_date"].apply(lambda x: timestampToSeason(x))

In [20]:
df

,Latitude,Longitude,birth_date,maxtempC,mintempC,totalSnow_cm,sunHour,tempC,windspeedKmph,winddirDegree,...,cloudcover,HeatIndexC,DewPointC,WindChillC,WindGustKmph,FeelsLikeC,uvIndex,sunrise,sunset,season
0,49.535120,-111.631584,2018-08-20 03:22:54,22,15,0.0,13.4,15.0,11.0,112.0,...,33.0,15.0,-1.0,15.0,22.0,15.0,1,05:25 AM,07:34 PM,summer
1,31.619696,-109.441666,2019-03-21 16:27:59,21,12,0.0,11.6,21.0,28.0,224.0,...,8.0,22.0,-8.0,21.0,33.0,21.0,1,06:20 AM,06:30 PM,spring
2,32.371845,-98.762950,2018-06-24 21:42:12,39,25,0.0,14.2,31.0,31.0,155.0,...,3.0,33.0,20.0,31.0,47.0,33.0,1,05:30 AM,07:46 PM,summer
3,30.327080,-92.548600,2018-03-24 09:10:42,26,17,0.0,8.8,23.0,25.0,205.0,...,35.0,25.0,17.0,23.0,31.0,25.0,1,06:09 AM,06:24 PM,spring
4,44.542300,-82.181320,2018-11-11 08:18:32,3,0,0.0,7.0,0.0,24.0,229.0,...,92.0,1.0,-3.0,-5.0,30.0,-5.0,1,07:20 AM,05:05 PM,autumn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,41.891644,-110.873830,2018-11-08 13:15:28,-1,-7,0.0,8.7,-2.0,24.0,293.0,...,19.0,-3.0,-12.0,-9.0,28.0,-9.0,1,07:05 AM,05:09 PM,autumn
458,38.141678,-101.836390,2019-04-04 15:45:12,20,7,0.0,12.9,19.0,8.0,112.0,...,4.0,19.0,3.0,19.0,9.0,19.0,1,05:28 AM,06:13 PM,spring
459,48.635270,-105.471466,2018-02-19 03:00:45,-17,-24,0.4,3.9,-24.0,5.0,359.0,...,82.0,-24.0,-25.0,-29.0,10.0,-29.0,1,07:02 AM,05:30 PM,winter
460,33.990128,-86.597240,2018-10-19 16:39:36,25,11,0.0,11.5,25.0,10.0,230.0,...,19.0,26.0,15.0,25.0,12.0,26.0,1,05:55 AM,05:08 PM,autumn


In [21]:
df.to_csv('./raw04.csv', index=False)